# Laboratório 4
Material desenvolvido por Henrique Margotte e Aurora Pozo para a disciplina de Aprendizado de Máquina, para o curso de Informática Biomédica da UFPR, semestre 2025/2. Códigos baseados em exemplos da biblioteca scikit-learn e nos notebooks Python do livro "Inteligência Artificial: Uma Abordagem de Aprendizado de Máquina", 2ª edição.

# Exemplo 1: Processamento de Linguagem Natural (PLN)
Nos últimos laboratórios, tratamos de diversas técnicas de pré-processamento de dados e aprendemos a como transformar atributos categóricos em formato textual para numérico, mas, como fazemos se os nossos dados são compostos de textos? Vamos pensar sobre isso observando o exemplo a seguir:

In [26]:
# Preparando o ambiente
import pandas as pd
import numpy as np

# Importando base de dados
df = pd.read_csv('title_reviews.csv')
df = df.tail(100) # usando apenas 100 amostras para exemplo
df.tail(10)

,class,review_title
3990,2,"multiple uses, ergonomic...elegant simplicity"
3991,1,Does not work with Dell Studio XPS 9000
3992,2,The BOLD truth
3993,2,Sweet Sleep - Just What I Needed
3994,1,I bought 30...17 are broken...
3995,1,Not as good smelling as I thought.
3996,1,Atomic (?) Clock
3997,1,Obee don't know me
3998,2,Quality Product
3999,2,Neil Diamond; The Ultimate Collection


Os dados acima são uma amostra da base "Amazon Review Polaridy Dataset", disponibilizada por Xiang Zhang (https://drive.google.com/file/d/0Bz8a_Dbh9QhbaW12WVVZS2drcnM/view?usp=drive_link&resourcekey=0-NlNEsGjcAEPJBwDJtuuZ3Q). Ela é composta de avaliações reais de produtos da Amazon, com a classe sendo atribuída como 1: avaliações negativas; e 2: avaliações positivas. A base ainda possui uma coluna com o título da avaliação e outra com o texto completo, mas que foi suprimida para esse exemplo.

Levando isso em conta, ela não é tão diferente das outras bases que utilizamos, possui uma classe definida e atributos que podem ser utilizados para classificar os dados, o que poderia ser ensinado a um modelo de Aprendizado de Máquina (AM). Mas, como podemos transformar esse texto em informação numérica para um modelo aprender?

Esse tipo de problema faz parte da área de Processamento de Linguagem Natural (PNL, ou NLP, do inglês Natural Language Processing). O PNL engloba diversos problemas relacionados com linguagem natural, ou seja, humana, desde textos escritos, até falas e língua de sinais. No Laboratório de hoje iremos explorar apenas uma amostra do PNL, mas a área é muito mais extensa do que podemos tratar aqui, incluindo sistemas de tradução automática, reconhecimento de fala, os grandes modelos de linguagem (LLMs), entre outros!

## Normalizando os dados
Uma das etapas de pré-processamento que podemos aplicar em textos, antes mesmo de transformá-los para conceitos numéricos, é a normalização. Note que essa normalização não é a mesma que aplicamos nos dados até agora, aqui, o objetivo é deixar o texto um pouco mais compreensível e simples para podermos extrair informações.

Um dos métodos mais simples de fazer isso, é mantendo todas as letras como minúsculas ou maiúsculas, assim o modelo poderá entender que "Exemplo" e "exemplo" são a mesma palavra. Podemos usar os métodos do tipo `str` (string) para isso, com a função `lower` transformando as strings para letras minúsculas e a função `strip` removendo, por padrão, espaços em branco extras, tabulações e quebras de linhas, que podem ser úteis dependendo das técnicas a serem utilizadas.

In [27]:
# normalize the text
df['review_title'] = df['review_title'].str.lower()
df['review_title'] = df['review_title'].str.strip()
df.tail(10)

,class,review_title
3990,2,"multiple uses, ergonomic...elegant simplicity"
3991,1,does not work with dell studio xps 9000
3992,2,the bold truth
3993,2,sweet sleep - just what i needed
3994,1,i bought 30...17 are broken...
3995,1,not as good smelling as i thought.
3996,1,atomic (?) clock
3997,1,obee don't know me
3998,2,quality product
3999,2,neil diamond; the ultimate collection


Isso ainda pode gerar conflitos, como por exemplo "rs", que normalemente se refere a risos, e "RS", sigla do Rio Grande do Sul, seriam tratados como a mesma palavra. Mas, mesmo em palavras sem essa diferença isso pode ocorrer, como na palavra "manga", que pode ser tanto uma fruta quanto uma parte de uma roupa, e existem métodos de contornar isso, como os mecanismos de atenção utilizados por LLMs, e que não abordaremos aqui.

Ainda há outros problemas que poderiam ser tratados nessa etapa, como aproximar palavras equivalentes, como "programador" e "programadora" em contextos em que o gênero não é relevante, ou conjugações de verbos, já que "correr", "corri", "correu" e "corremos" podem ter significados bem próximos, mas que serão tratados diferentemente a depender das técnicas. A tokenização é uma estratégia para contornar esses casos, por exemplo.

Por enquanto, manteremos nossa base de dados do jeito que está, apenas aplicando as funções `lower` e `strip`.

## Removendo palavras irrelevantes
Assim como removemos atributos irrelevantes para o problema nos laboratórios anteriores, podemos excluir palavras que não agregam para o problema que estamos tentando resolver. Por exemplo, os conectivos, preposições e artigos ("e", "com", "as", "o", "de", "para", etc.) podem não ser relevantes. Esses tipos de palavras são chamadas de *stopwords*, e algumas bibliotecas já incluem dicionários e funções próprias para removê-las do texto. Também podem ser removidas palavras manualmente, adicionando-as à lista de *stopwords*, para casos em que uma palavra seja irrelevante em seu contexto, como a palavra "livro" em uma base de dados de avaliações de uma biblioteca.

In [28]:
# remove stop words
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

df['review_title'].apply(lambda x: ' '.join([word for word in x.split() if word not in ENGLISH_STOP_WORDS])).tail(10)

,review_title
3990,"multiple uses, ergonomic...elegant simplicity"
3991,does work dell studio xps 9000
3992,bold truth
3993,sweet sleep - just needed
3994,bought 30...17 broken...
3995,good smelling thought.
3996,atomic (?) clock
3997,obee don't know
3998,quality product
3999,neil diamond; ultimate collection


## Vetorizando
Até agora, falamos apenas de tratar o texto, mas não o transformamos em algo numérico para que o modelo possa entender corretamente. Para fazer isso, vamos utilizar técnicas de vetorização.

Vetorização é o processo de converter as palavras em vetores de números. Nesse Laboratório, abordaremos duas técnicas, cada uma com sua função no scikit-learn:
- `CountVectorizer`: Esse método cria uma matriz de frequência, em que cada coluna representa uma palavra do vocabulário e cada linha o texto da base, com os valores representando a quantidade de vezes que a palavra aparece nesse documento. Com isso, palavras que aparecem mais em um mesmo texto, teriam valores maiores, sendo um comportamento mais perceptível em textos maiores.
- `TfidfVectorizer`: O TF-IDF (*Term Frequency-Inverse Document Frequency*) é uma técnica semelhante à anterior. TF é a frequência do termo, calculada pela quantidade de vezes que a palavra aparece no documento, dividida pela quantidade de termos do documento. Já IDF é a frequência inversa no documento, medida através da quantidade de vezes que essa palavra aparece em outros documentos, diminuindo os valores de palavras que sejam muito comuns em todos os textos, como as *stopwords*.

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['review_title'].tail(10))
print(vectorizer.get_feature_names_out())
print(X.toarray())

['17' '30' '9000' 'are' 'as' 'atomic' 'bold' 'bought' 'broken' 'clock'
 'collection' 'dell' 'diamond' 'does' 'don' 'elegant' 'ergonomic' 'good'
 'just' 'know' 'me' 'multiple' 'needed' 'neil' 'not' 'obee' 'product'
 'quality' 'simplicity' 'sleep' 'smelling' 'studio' 'sweet' 'the'
 'thought' 'truth' 'ultimate' 'uses' 'what' 'with' 'work' 'xps']
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 1 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
  0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0
  0 0 1 0 0 0]
 [1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0
  0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['review_title'].tail(10))
print(vectorizer.get_feature_names_out())
print(X.toarray())

['17' '30' '9000' 'are' 'as' 'atomic' 'bold' 'bought' 'broken' 'clock'
 'collection' 'dell' 'diamond' 'does' 'don' 'elegant' 'ergonomic' 'good'
 'just' 'know' 'me' 'multiple' 'needed' 'neil' 'not' 'obee' 'product'
 'quality' 'simplicity' 'sleep' 'smelling' 'studio' 'sweet' 'the'
 'thought' 'truth' 'ultimate' 'uses' 'what' 'with' 'work' 'xps']
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.4472136  0.4472136  0.
  0.         0.         0.         0.4472136  0.         0.
  0.         0.         0.         0.         0.4472136  0.
  0.         0.         0.         0.         0.         0.
  0.         0.4472136  0.         0.         0.         0.        ]
 [0.         0.         0.359846   0.         0.         0.
  0.         0.         0.         0.         0.         0.359846
  0.         0.359846   0.         0.         0.         0.
  0.         0.         0.         0.   

Nas matrizes acima, conseguimos ver a conversão dos textos para valores numéricos. Há outros meios de se fazer isso, como os *embeddings*, por exemplo, mas nos manteremos nessas técnicas para a atividade de hoje.

Caso queira conhecer mais sobre PLN, o livro "Processamento de Linguagem Natural: Conceitos, Técnicas e Aplicações em Português", disponível gratuitamente em https://brasileiraspln.com/livro-pln/, apresenta diversos conceitos de PNL de forma aprofundada e em português!

# Exemplo 2: O algoritmo Naive Bayes
Agora que sabemos um pouco mais sobre como transformar texto em valores numéricos compreensíveis para um modelo de AM, vamos voltar para os modelos! O Naive Bayes é um algoritmo simples, baseado em probabilidades, que considera a interdependência entre os atributos. Ele pode ser utilizado para classificação de texto, levando em conta que cada palavra impacta na probabilidade daquele texto pertencer àquela classe.

Como exemplo, aplicaremos o algoritmo através do classificador `MultinomialNB` do scikit-learn, que implementa o Naive Bayes para o formato de entrada que extraímos do texto. Há outros classificadores para valores categóricos, booleanos, entre outros.

Nosso objetivo será identificar se o título da avaliação se refere a uma avaliação positiva ou negativa, somente através das palavras utilizadas. Esse tipo de classificação faz parte de um problema chamado de Análise de Sentimento, sendo apenas uma das abordagens utilizadas para tal.

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['review_title'])
X_train, X_test, y_train, y_test = train_test_split(X,df['class'],random_state=42)

model = MultinomialNB()
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred))

0.52
              precision    recall  f1-score   support

           1       0.25      0.25      0.25         8
           2       0.65      0.65      0.65        17

    accuracy                           0.52        25
   macro avg       0.45      0.45      0.45        25
weighted avg       0.52      0.52      0.52        25



E nosso modelo teve 52% de acurácia, o que seria quase o mesmo de prever aleatoriamente!

Mas, por quê isso? Em primeiro lugar, esse foi apenas um caso de exemplo, utilizando 100 avaliações, o que pode não ser suficiente para treinar o modelo. Mas, compreender exatamente o comportamento do modelo não é tão simples, já que temos muitas dimensões (cada palavra na base de dados é uma dimensão), o que dificulta usarmos gráficos para apresentar os dados. Algumas técnicas de IA Explicável (XAI), como o SHAP, poderiam auxiliar a compreender as decisões do modelo. Isso não será aborado hoje, mas soube que haverá uma palestra na SACI (Semana Acadêmica de Computação e Informática) sobre isso 👀!

Por enquanto, vamos nos concentrar no que já conhecemos! Temos como melhorar esse resultado? Podemos tentar!

Ou melhor... *você* pode!

# Exercício!
SUA VEZ!

Vamos tentar melhorar nosso Naive Bayes e ver como as técnicas de PLN que aprendemos se comportam com ele!

Suas tarefas são as seguintes:
- Importe a base completa de `title_reviews.csv` fornecida (No Exemplo 1, utilizamos apenas 100 linhas, das 2000 presentes)
- Implemente o algoritmo Naive Bayes para a base completa, variando as seguintes etapas:
  - A técnica de vetorização utilizada entre as duas apresentadas
  - A remoção ou não de *stopwords*
- Compare os resultados obtidos e defina qual configuração performou melhor

In [ ]:
# Seu código aqui!

# DESAFIO! (Opcional)
Apenas o título das avaliações pode não ser suficiente para a classificação destas. Tente replicar o exercício utilizando a base `text_reviews.csv`, que possui o texto completo das avaliações. Avalie também quais as diferenças entre as bases de dados, se a vetorização se comporta de maneira semelhante ou não, etc!